Ниже приведен код без результатов его запуска, т.к. обучение моделей на исходных данных и подбор параметров на доступном мне оборудовании занимает много времени

# Классификатор объявлений по категориям на основе сверточной нейронной сети

Описание метода приведено в статье "Convolutional Neural Networks for Sentence Classification"
http://arxiv.org/pdf/1408.5882v2.pdf

Еще одна статья об использовании CNN в обработке текстов:
http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/

Релизация: https://github.com/alexander-rakhlin/CNN-for-Sentence-Classification-in-Keras/

Импортируем основные библиотеки для анализа данных

In [ ]:
import pandas as pd
import numpy as np

Зафиксируем генератор случайных чисел для воспроизводимости результатов

In [ ]:
np.random.seed(2)

Загрузим исходные данные и просмотрим первые записи

In [ ]:
train = pd.read_csv('C:\\Users\\al.nikolaev\\Desktop\\Avito\\data\\train.csv')

In [ ]:
train.head()

### Предобработаем тексты перед их анализом

In [ ]:
import re # регулярные выражения

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer() # морфологический анализатор русского языка

In [ ]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian') # загружаем набор стоп-слов русского языка (самые частотные, не определяют тему)
stop_words += [u'купить', u'продать', u'отдать', u'цена', u'метро', u'недорого', u'дешево', u'продаваться', u'новый', 
               u'самовывоз', u'адрес'] # добавляем слова, типичные для объявлений, но не определяющие тему

In [ ]:
def normalize_text(text):
    text = text.decode('utf-8') # декодируем юникод
    text = re.sub(r"\s+[^\s]*[0-9]+[^\s]*([\s]+|$)", " ", text) # убираем модели, номера, измерения
    text = re.sub(ur"[^ЁёА-Яа-яA-Za-z\s]", " ", text) # убираем все небуквенные символы
    text = re.sub(r"\s{2,}", " ", text) # убираем лишние пробелы
    text = text.strip() # убираем пробелы в начале и в конце
    text = text.lower() # приводим все слова к строчным буквам
    text = text.split(" ") # разбиваем текст на слова
    text = [morph.parse(word)[0].normal_form for word in text] # заменяем каждое слово его нормальной формой
    text2 = []
    for word in text:
        if len(word)>1 and not word in stop_words:
            text2.append(word) # исключаем из текстов стоп-слова и слова длины 1
    text = text2
    return text2

Создаем набор нормализованных и разбитых на слова текстов

In [ ]:
texts = []

for i in range(len(train)):
    text = train.title[i] + ' ' + train.description[i] 
    texts.append(normalize_text(text))

Дополняем все тексты так, чтобы они содержали одинаковое количество слов (необходимо для обучения нейронной сети)

In [ ]:
def equalize_texts(texts, eq_word="<eq>"):
    max_len = max(len(x) for x in texts)
    eq_texts = []
    for i in range(len(texts)):
        text = texts[i]
        num_to_add = max_len - len(text)
        text += [eq_word] * num_to_add
        eq_texts.append(text)
    return eq_texts

In [ ]:
texts = equalize_texts(texts)

Создадим словарь всех слов текстов и упорядочим их по частоте

In [ ]:
import itertools
from collections import Counter

In [ ]:
def build_vocab(texts):
    word_counts = Counter(itertools.chain(*texts)) # счетчик встречаемости слов во всем корпусе текстов
    # Словарь для поиска слова по индексу
    vocabulary_inv = [x[0] for x in word_counts.most_common()] # записываем слова по убыванию частоты встречаемости
    # Словарь для поиска индекса по слову
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]

In [ ]:
vocab, vocab_inv = build_vocab(texts)

### Для векторизации слов текстов и сохранения их смысловой нагрузки используем предобученные на корпусе русских текстов векторы Word2Vec

Используем готовую функцию для загрузки предобученного на корпусе русских текстов Word2Vec (из документации к файлу)

In [ ]:
import logging

from numpy import zeros, dtype, float32 as REAL, fromstring
from gensim.models.word2vec import Vocab
from gensim import utils
from gensim.models.word2vec import Word2Vec

logger = logging.getLogger("gensim.models.word2vec")

def load_vectors(fvec):
#    return gs.models.Word2Vec.load_word2vec_format(fvec,binary=True)
    return load_word2vec_format(fvec, binary=True)


def load_word2vec_format(fname, fvocab=None, binary=False, norm_only=True, encoding='utf8'):
    counts = None
    if fvocab is not None:
        logger.info("loading word counts from %s" % (fvocab))
        counts = {}
        with utils.smart_open(fvocab) as fin:
            for line in fin:
                word, count = utils.to_unicode(line).strip().split()
                counts[word] = int(count)

    logger.info("loading projection weights from %s" % (fname))
    with utils.smart_open(fname) as fin:
        header = utils.to_unicode(fin.readline(), encoding=encoding)
        vocab_size, vector_size = map(int, header.split())  # throws for invalid file format
        result = Word2Vec(size=vector_size)
        result.syn0 = zeros((vocab_size, vector_size), dtype=REAL)
        if binary:
            binary_len = dtype(REAL).itemsize * vector_size
            for line_no in xrange(vocab_size):
                # mixed text and binary: read text first, then binary
                word = []
                while True:
                    ch = fin.read(1)
                    if ch == b' ':
                        break
                    if ch != b'\n':  # ignore newlines in front of words (some binary files have)

                        word.append(ch)
                try:
                    word = utils.to_unicode(b''.join(word), encoding=encoding)
                except UnicodeDecodeError, e:
                    logger.warning("Couldn't convert whole word to unicode: trying to convert first %d bytes only ..." % e.start)
                    word = utils.to_unicode(b''.join(word[:e.start]), encoding=encoding)
                    logger.warning("... first %d bytes converted to '%s'" % (e.start, word))

                if counts is None:
                    result.vocab[word] = Vocab(index=line_no, count=vocab_size - line_no)
                elif word in counts:
                    result.vocab[word] = Vocab(index=line_no, count=counts[word])
                else:
                    logger.warning("vocabulary file is incomplete")
                    result.vocab[word] = Vocab(index=line_no, count=None)
                result.index2word.append(word)
                result.syn0[line_no] = fromstring(fin.read(binary_len), dtype=REAL)
        else:
            for line_no, line in enumerate(fin):
                parts = utils.to_unicode(line[:-1], encoding=encoding).split(" ")
                if len(parts) != vector_size + 1:
                    raise ValueError("invalid vector on line %s (is this really the text format?)" % (line_no))
                word, weights = parts[0], list(map(REAL, parts[1:]))
                if counts is None:
                    result.vocab[word] = Vocab(index=line_no, count=vocab_size - line_no)
                elif word in counts:
                    result.vocab[word] = Vocab(index=line_no, count=counts[word])
                else:
                    logger.warning("vocabulary file is incomplete")
                    result.vocab[word] = Vocab(index=line_no, count=None)
                result.index2word.append(word)
                result.syn0[line_no] = weights
    logger.info("loaded %s matrix from %s" % (result.syn0.shape, fname))
    result.init_sims(norm_only)
    return result

Загрузим предобученные векторы word2vec (источник: https://github.com/nlpub/russe-evaluation/tree/master/russe/measures/word2vec)

In [ ]:
word_vecs = load_vectors('C:\\Users\\al.nikolaev\\Desktop\\Avito\\word2vec\\rus100.w2v')

Размерность каждого вектора 1x100

In [ ]:
len(word_vecs[u'кошка'])

Отберем векторы Word2Vec для слов, которые есть в словаре, остальные заменим случайными векторами (векторизуем словарь)

In [ ]:
embedding_weights = [np.array([word_vecs[w] if w in word_vecs else np.random.uniform(-0.25,0.25,100) for w in vocab_inv])]

### Подготовим входные данные для модели

Закодируем цифрами индекса слова в корпусе текстов, чтобы для них можно было по индексу получать векторы из векторизованного словаря

In [ ]:
x = np.array([[vocab[word] for word in text] for text in texts])

Целевая переменная - номер категории

In [ ]:
y = train.category_id

Перемешаем исходные данные перед обучением

In [ ]:
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

### Построим сверточную нейронную сеть с помощью библиотеки keras

Зададим параметры модели

In [ ]:
# Параметры исходных данных
sequence_length = 455 # длина каждого текста выровнена до 455
embedding_dim = 100 # размерность векторов word2vec         

# Гиперпараметры модели 
filter_sizes = (3, 4, 5)
num_filters = 5
dropout_prob = (0.5, 0.7)
hidden_dims = 100

# Параметры обучения
batch_size = 50
num_epochs = 100
val_split = 0.1

Зададим структуру нейронной сети

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Input, Merge, Convolution1D, MaxPooling1D

# Создаем слои свертки и подвыборки, а также выходной слой
graph_in = Input(shape=(sequence_length, embedding_dim))
convs = []
for fsz in filter_sizes:
    conv = Convolution1D(nb_filter=num_filters,
                         filter_length=fsz,
                         border_mode='valid', # сужающая свертка
                         activation='relu',
                         subsample_length=1)(graph_in)
    pool = MaxPooling1D()(conv)
    flatten = Flatten()(pool)
    convs.append(flatten)
    
out = Merge(mode='concat')(convs)
graph = Model(input=graph_in, output=out)

# Основная структура модели (последовательность слоев)
model = Sequential()
model.add(Embedding(embedding_weights[0].shape[0], embedding_dim, input_length=sequence_length, weights=embedding_weights))
model.add(Dropout(dropout_prob[0], input_shape=(sequence_length, embedding_dim))) # dropout отключает часть нейронов, 
# чтобы избежать их подстройки друг под друга и переобучения
model.add(graph)
model.add(Dense(hidden_dims))
model.add(Dropout(dropout_prob[1]))
model.add(Activation('relu')) # функция активации ReLu max(0,x)
model.add(Dense(1))
model.add(Activation('sigmoid')) # функция активации сигмоида
model.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['accuracy']) # метрика качества accuracy
# цель - минимизация средней абсолютной ошибки, т.к. близкие друг к другу категории чаще всего близки по id

### Обучим модель

In [ ]:
model.fit(x_shuffled, y_shuffled, batch_size=batch_size, nb_epoch=num_epochs, validation_split=val_split, verbose=2)

# Альтернативная, более простая модель - SVM

Описание и реализация метода приведены в статье "Supervised Learning for Document Classification with Scikit Learn"
https://www.quantstart.com/articles/Supervised-Learning-for-Document-Classification-with-Scikit-Learn

Подготовим исходные данные: закодируем слова текстов методом TF-IDF.

При преобразовании TF-IDF вес слова пропорционален количеству употребления этого слова в документе и обратно пропорционален частоте употребления слова в других документах коллекции. Каждому документу будет соответствовать отдельная строка, каждому слову - отдельный столбец. Данных подход позволяет не учитывать фоновые, часто встречающиеся и не определяющие тему слова.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1)
x_tfidf = vectorizer.fit_transform([' '.join(text) for text in texts[shuffle_indices]])
y_tfidf = y[shuffle_indices]

Используем модель SVM с радиальным ядром

In [ ]:
from sklearn.svm import SVC
svm = SVC(C=1000000.0, gamma='auto', kernel='rbf') 
# C - штраф за помещение перемешанных объектов разных классов в промежуточную полосу

Обучим модель

In [ ]:
svm.fit(x_tfidf, y_tfidf)

# Оценка качества моделей на тестовых данных

Подготовка тестовых данных

In [ ]:
test = pd.read_csv('C:\\Users\\al.nikolaev\\Desktop\\Avito\\data\\train.csv')

В функцию нормализации добавляем условие: оставлять только слова из тренировочного корпуса текстов

In [ ]:
def normalize_text(text):
    text = text.decode('utf-8') # декодируем юникод
    text = re.sub(r"\s+[^\s]*[0-9]+[^\s]*([\s]+|$)", " ", text) # убираем модели, номера, измерения
    text = re.sub(ur"[^ЁёА-Яа-яA-Za-z\s]", " ", text) # убираем все небуквенные символы
    text = re.sub(r"\s{2,}", " ", text) # убираем лишние пробелы
    text = text.strip() # убираем пробелы в начале и в конце
    text = text.lower() # приводим все слова к строчным буквам
    text = text.split(" ") # разбиваем текст на слова
    text = [morph.parse(word)[0].normal_form for word in text] # заменяем каждое слово его нормальной формой
    text2 = []
    for word in text:
        if word in vocab: # добавляем только слова из словаря
            text2.append(word)
    text = text2
    return text2

In [ ]:
texts = []

for i in range(len(test)):
    text = test.title[i] + ' ' + test.description[i] 
    texts.append(normalize_text(text))

In [ ]:
texts = equalize_texts(texts)

In [ ]:
x_test = np.array([[vocab[word] for word in text] for text in texts])
x_tfidf_test = vectorizer.transform([' '.join(text) for text in texts)
y_test = test.category_id

In [ ]:
from sklearn.metrics import accuracy_score

y_cnn = model.predict(x_test)
y_svm = svm.predict(x_tfidf_test)

cnn_score = accuracy_score(y_test, y_cnn)
svm_score = accuracy_score(y_test, y_svm)

print cnn_score, svm_score

Рассчитаем точность по категориям

In [ ]:
cat = pd.read_csv('C:\\Users\\al.nikolaev\\Desktop\\Avito\\data\\category.csv')

Добавим столбцы с первым и вторым уровнем категорий

In [ ]:
h = []
for t in cat.name:
    h.append(t.split("|"))
    
cat0, cat1 = [], []
for i in range(len(h)):
    cat0.append(h[i][0])
    cat1.append(h[i][1])
    
cat_split = pd.DataFrame(zip(cat0, cat1))
cat = pd.concat([cat, cat_split], axis=1)
cat.columns = ['category_id', 'name', 'cat0', 'cat1']

Создадим датафреймы с истинными категориями тестовой выборки и предсказанными значениями

In [ ]:
df_true = pd.DataFrame(y_test)
df_cnn = pd.DataFrame(y_cnn)
df_svm = pd.DataFrame(y_svm)

df_true.columns = ['category_id']
df_cnn.columns = ['category_id']
df_svm.columns = ['category_id']

Соединим их с таблицей категорий

In [ ]:
df_true = df_true.merge(cat, on='category_id', how='left')
df_cnn = df_cnn.merge(cat, on='category_id', how='left')
df_svm = df_svm.merge(cat, on='category_id', how='left')

Оценим точность классификации по верхнему уровню категорий

In [ ]:
acc0_cnn = np.array([1 if a == b else 0 for a, b in zip(df_true.cat0, df_cnn.cat0)]).mean()
acc0_svm = np.array([1 if a == b else 0 for a, b in zip(df_true.cat0, df_svm.cat0)]).mean()

print acc0_cnn, acc0_svm

In [ ]:
acc1_cnn = np.array([1 if a == b else 0 for a, b in zip(df_true.cat1, df_cnn.cat1)]).mean()
acc1_svm = np.array([1 if a == b else 0 for a, b in zip(df_true.cat1, df_svm.cat1)]).mean()

print acc1_cnn, acc1_svm